In [1]:
#This code takes as an input a folder of 2D tif masks, it transforms them into a stack, binarizes 
#them, applies a median z-filter to correct skipped frames in the glomeruli segmentation, applies 
#connected components analysis and gives a tif stack and a txt file with glomeruli data as output.

#For large images (Z = 4160) coming from two different cameras (CAM1, CAM2) a different image is saved
#for each camera (otherwise it required too much RAM).

import os

#Folder
imPath = 'D:/AAV para enfermedades renales/Stardist glomeruli/Stardist results on whole image/' + \
'Result on large image/Kidney1_derecho_4.8x_1x3_R&Lshifted1.6mm/RL01--X00--Y03--C01'

#File
#resultName = 'Corrected RL01--X00--Y03--C01 mask'
resultName = 'trialMarchingCubes'

visualSlide = 250

#FilterSize (dimensions [Z,Y,X]). The filter analyzes the pixel in its middle
filtSize = [5,1,1]

#Check whether you want to perform connected component analysis or not
conComps = True


#The order in which you want to process both cameras
cams = ['CAM1','CAM2']

#Whether you want only the first cam in "cams" to be processed (because of memory limits)
processOnlyOneCam = True
#------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------




from glob import glob as glob
import numpy as np
import tifffile as tiff
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage import measure
import time

tic0 = time.time()
tic=tic0

imageNamesOrig = glob(imPath+'/*.tif')


      
for CAM in cams:

    #Keep only the names for 'CAM1' or 'CAM2'
    imageNames = [name for name in imageNamesOrig if CAM in name]

    imShape = tiff.imread(imageNames[0]).shape
    numIms = len(imageNames)

    stackDims = np.append(numIms,imShape)

    stack = np.empty (stackDims,dtype = np.uint16)

    slide = -1

    print('\nConverting ',stackDims[0], ' single (',stackDims[1],' x ',stackDims[2], \
          ') yx slides into a zyx stack\n')

    #Save all images from the folder as slides of "stack"
    for imageName in imageNames:
        slide+=1
        im = tiff.imread(imageName)
        #Binarize the image
        im = 1*(im>0)

        #Change bit depth
        im.astype('uint8')
        stack[slide,:,:] = im
        print('Slide ',slide,' of ',stackDims[0],' added')

    #plt.figure()
    #plt.imshow(stack[visualSlide,:,:])
    #plt.title('Binarized stack before z-filtering')    

    #Binarize the stack
    #stack = 1*(stack>0)
    #print('The stack has been binarized')

    #Apply the z-median filter (yz dimensions )
    stack = ndimage.median_filter(stack,size = filtSize)

    print('\nThe z-median filter (shape ',str(filtSize),' has been applied')
    toc = time.time()
    timeStep=toc-tic
    print('Time Z filter: ',timeStep)
    tic = toc
    #plt.figure()
    #plt.imshow(stack[visualSlide,:,:])
    #plt.title('Stack after z-filtering')

    if conComps is True:
        #Extract the connected components from the binary stack

        if (type(stack[0,0,0]) is not 'uint8') and (np.max(stack)<=(2**16-1)):
            stack = stack.astype('uint16')

        print(type(stack[0,0,0]))
        stack = measure.label(stack)

        print('\nThe connected components have been labeled')
        toc = time.time()
        timeStep = toc-tic
        print('Time connected components labelling: ',timeStep)
        tic = toc
        
        conCompData = measure.regionprops(stack) 
        
        print('\nThe connected components have been analyzed')
        toc = time.time()
        timeStep=toc-tic
        print('Time connected components analysis: ',timeStep)
        tic = toc
        
        
    if (type(stack[0,0,0]) is not 'uint8') and (np.max(stack)<=(2**16-1)):
        stack = stack.astype('uint16')

    #Only if there are different cams save the CAM number in the name
    if 'CAM' in imageNamesOrig[0]:
        resultPath = os.path.dirname(imPath) + str(CAM) + '-' + \
        resultName + '-z-filtered(dimsZ'+str(filtSize[0])+'Y'+str(filtSize[1])+'X'+str(filtSize[2])

    else:
        resultPath = os.path.dirname(imPath) +  \
        resultName + '-z-filtered(dimsZ'+str(filtSize[0])+'Y'+str(filtSize[1])+'X'+str(filtSize[2])

    if conComps is True:
        resultPath += '-conComps.tif'
    else:
        resultPath +='.tif'    
        
    tiff.imsave(resultPath, stack, bigtiff=True)
    
    #Free some memory by deleting the stack variable
    
    
    print('\n',str(CAM),' stack has been saved')
    
    #Save the connected components Data:
    #----------------------------------------------------------------
    features = list()
    #First, create the features vector
    for feature in conCompData[0]:
        features.append(feature)

    features.remove('convex_image')
    features.remove('coords')
    features.remove('filled_image')
    features.remove('image')
    features.remove('inertia_tensor')
    features.remove('moments')
    features.remove('moments_central')
    features.remove('moments_normalized')
    features.remove('eccentricity')
    features.remove('moments_hu')
    features.remove('orientation')
    features.remove('perimeter')
    features.remove('perimeter_crofton')
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    #Introduce the marching cubes algorithm:
    surfaces = np.zeros(np.max(stack))
    
    #Trying to analyze one surface at a time (for each glomerulus)
    for i in range(1,np.max(stack)+1):
        verts, faces, _, _  = measure.marching_cubes(stack,i)
    
        surfaces[i-1]= measure.mesh_surface_area(verts, faces)        
    print(surfaces)
    print('surfaces.shape: ',surfaces.shape)
    print('\nMarching cubes have been applied')
    toc = time.time()
    timeStep=toc-tic
    print('Time connected components analysis: ',timeStep)
    toc = tic
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    del stack
    
    if 'CAM' in imageNamesOrig[0]:
        conCompDataPath = os.path.dirname(imPath) + str(CAM) + '-' +  \
        'conCompData'+ resultName +'.txt'
    else:
        conCompDataPath = os.path.dirname(imPath) +  \
        'conCompData'+ resultName + '.txt'
    
    with open(conCompDataPath, 'w') as f:
        print('\nWriting connected components data in a .txt file')
        for feature in features:
            f.write(feature)
            f.write(';')
        f.write('\n')
        for component in conCompData:
            for feature in features:
                try:
                    f.write(str(component[feature]))
                except:
                    f.write(' ')
                f.write(';')
            f.write('\n')
    print('\nThe .txt with the connected components data has been saved')
    toc =time.time()
    timeStep = toc-tic
    print('Time writing connected components data in .txt',timeStep,'\n\n')
    
    tic = toc
    #Try to free as much memory as possible
    del feature, features,conCompDataPath
    #-------------------------------------------------------------
    
    #Do not perform the second iteration if there are no cameras in the images or if you only want
    #to process the first camera in "cams"
    if ('CAM' not in imageNamesOrig[0]) or (processOnlyOneCam is True):
        break
        
toc = time.time()        
timeStep = toc-tic0
print('\nTotal execution time: ',timeStep)

c:\users\equipo\.conda\envs\workenvpy3.7\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\equipo\.conda\envs\workenvpy3.7\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\users\equipo\.conda\envs\workenvpy3.7\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)



Converting  2004  single ( 2048  x  2048 ) yx slides into a zyx stack

Slide  0  of  2004  added
Slide  1  of  2004  added
Slide  2  of  2004  added
Slide  3  of  2004  added
Slide  4  of  2004  added
Slide  5  of  2004  added
Slide  6  of  2004  added
Slide  7  of  2004  added
Slide  8  of  2004  added
Slide  9  of  2004  added
Slide  10  of  2004  added
Slide  11  of  2004  added
Slide  12  of  2004  added
Slide  13  of  2004  added
Slide  14  of  2004  added
Slide  15  of  2004  added
Slide  16  of  2004  added
Slide  17  of  2004  added
Slide  18  of  2004  added
Slide  19  of  2004  added
Slide  20  of  2004  added
Slide  21  of  2004  added
Slide  22  of  2004  added
Slide  23  of  2004  added
Slide  24  of  2004  added
Slide  25  of  2004  added
Slide  26  of  2004  added
Slide  27  of  2004  added
Slide  28  of  2004  added
Slide  29  of  2004  added
Slide  30  of  2004  added
Slide  31  of  2004  added
Slide  32  of  2004  added
Slide  33  of  2004  added
Slide  34  of  2004 

Slide  295  of  2004  added
Slide  296  of  2004  added
Slide  297  of  2004  added
Slide  298  of  2004  added
Slide  299  of  2004  added
Slide  300  of  2004  added
Slide  301  of  2004  added
Slide  302  of  2004  added
Slide  303  of  2004  added
Slide  304  of  2004  added
Slide  305  of  2004  added
Slide  306  of  2004  added
Slide  307  of  2004  added
Slide  308  of  2004  added
Slide  309  of  2004  added
Slide  310  of  2004  added
Slide  311  of  2004  added
Slide  312  of  2004  added
Slide  313  of  2004  added
Slide  314  of  2004  added
Slide  315  of  2004  added
Slide  316  of  2004  added
Slide  317  of  2004  added
Slide  318  of  2004  added
Slide  319  of  2004  added
Slide  320  of  2004  added
Slide  321  of  2004  added
Slide  322  of  2004  added
Slide  323  of  2004  added
Slide  324  of  2004  added
Slide  325  of  2004  added
Slide  326  of  2004  added
Slide  327  of  2004  added
Slide  328  of  2004  added
Slide  329  of  2004  added
Slide  330  of  2004

Slide  589  of  2004  added
Slide  590  of  2004  added
Slide  591  of  2004  added
Slide  592  of  2004  added
Slide  593  of  2004  added
Slide  594  of  2004  added
Slide  595  of  2004  added
Slide  596  of  2004  added
Slide  597  of  2004  added
Slide  598  of  2004  added
Slide  599  of  2004  added
Slide  600  of  2004  added
Slide  601  of  2004  added
Slide  602  of  2004  added
Slide  603  of  2004  added
Slide  604  of  2004  added
Slide  605  of  2004  added
Slide  606  of  2004  added
Slide  607  of  2004  added
Slide  608  of  2004  added
Slide  609  of  2004  added
Slide  610  of  2004  added
Slide  611  of  2004  added
Slide  612  of  2004  added
Slide  613  of  2004  added
Slide  614  of  2004  added
Slide  615  of  2004  added
Slide  616  of  2004  added
Slide  617  of  2004  added
Slide  618  of  2004  added
Slide  619  of  2004  added
Slide  620  of  2004  added
Slide  621  of  2004  added
Slide  622  of  2004  added
Slide  623  of  2004  added
Slide  624  of  2004

Slide  883  of  2004  added
Slide  884  of  2004  added
Slide  885  of  2004  added
Slide  886  of  2004  added
Slide  887  of  2004  added
Slide  888  of  2004  added
Slide  889  of  2004  added
Slide  890  of  2004  added
Slide  891  of  2004  added
Slide  892  of  2004  added
Slide  893  of  2004  added
Slide  894  of  2004  added
Slide  895  of  2004  added
Slide  896  of  2004  added
Slide  897  of  2004  added
Slide  898  of  2004  added
Slide  899  of  2004  added
Slide  900  of  2004  added
Slide  901  of  2004  added
Slide  902  of  2004  added
Slide  903  of  2004  added
Slide  904  of  2004  added
Slide  905  of  2004  added
Slide  906  of  2004  added
Slide  907  of  2004  added
Slide  908  of  2004  added
Slide  909  of  2004  added
Slide  910  of  2004  added
Slide  911  of  2004  added
Slide  912  of  2004  added
Slide  913  of  2004  added
Slide  914  of  2004  added
Slide  915  of  2004  added
Slide  916  of  2004  added
Slide  917  of  2004  added
Slide  918  of  2004

Slide  1171  of  2004  added
Slide  1172  of  2004  added
Slide  1173  of  2004  added
Slide  1174  of  2004  added
Slide  1175  of  2004  added
Slide  1176  of  2004  added
Slide  1177  of  2004  added
Slide  1178  of  2004  added
Slide  1179  of  2004  added
Slide  1180  of  2004  added
Slide  1181  of  2004  added
Slide  1182  of  2004  added
Slide  1183  of  2004  added
Slide  1184  of  2004  added
Slide  1185  of  2004  added
Slide  1186  of  2004  added
Slide  1187  of  2004  added
Slide  1188  of  2004  added
Slide  1189  of  2004  added
Slide  1190  of  2004  added
Slide  1191  of  2004  added
Slide  1192  of  2004  added
Slide  1193  of  2004  added
Slide  1194  of  2004  added
Slide  1195  of  2004  added
Slide  1196  of  2004  added
Slide  1197  of  2004  added
Slide  1198  of  2004  added
Slide  1199  of  2004  added
Slide  1200  of  2004  added
Slide  1201  of  2004  added
Slide  1202  of  2004  added
Slide  1203  of  2004  added
Slide  1204  of  2004  added
Slide  1205  o

Slide  1455  of  2004  added
Slide  1456  of  2004  added
Slide  1457  of  2004  added
Slide  1458  of  2004  added
Slide  1459  of  2004  added
Slide  1460  of  2004  added
Slide  1461  of  2004  added
Slide  1462  of  2004  added
Slide  1463  of  2004  added
Slide  1464  of  2004  added
Slide  1465  of  2004  added
Slide  1466  of  2004  added
Slide  1467  of  2004  added
Slide  1468  of  2004  added
Slide  1469  of  2004  added
Slide  1470  of  2004  added
Slide  1471  of  2004  added
Slide  1472  of  2004  added
Slide  1473  of  2004  added
Slide  1474  of  2004  added
Slide  1475  of  2004  added
Slide  1476  of  2004  added
Slide  1477  of  2004  added
Slide  1478  of  2004  added
Slide  1479  of  2004  added
Slide  1480  of  2004  added
Slide  1481  of  2004  added
Slide  1482  of  2004  added
Slide  1483  of  2004  added
Slide  1484  of  2004  added
Slide  1485  of  2004  added
Slide  1486  of  2004  added
Slide  1487  of  2004  added
Slide  1488  of  2004  added
Slide  1489  o

Slide  1739  of  2004  added
Slide  1740  of  2004  added
Slide  1741  of  2004  added
Slide  1742  of  2004  added
Slide  1743  of  2004  added
Slide  1744  of  2004  added
Slide  1745  of  2004  added
Slide  1746  of  2004  added
Slide  1747  of  2004  added
Slide  1748  of  2004  added
Slide  1749  of  2004  added
Slide  1750  of  2004  added
Slide  1751  of  2004  added
Slide  1752  of  2004  added
Slide  1753  of  2004  added
Slide  1754  of  2004  added
Slide  1755  of  2004  added
Slide  1756  of  2004  added
Slide  1757  of  2004  added
Slide  1758  of  2004  added
Slide  1759  of  2004  added
Slide  1760  of  2004  added
Slide  1761  of  2004  added
Slide  1762  of  2004  added
Slide  1763  of  2004  added
Slide  1764  of  2004  added
Slide  1765  of  2004  added
Slide  1766  of  2004  added
Slide  1767  of  2004  added
Slide  1768  of  2004  added
Slide  1769  of  2004  added
Slide  1770  of  2004  added
Slide  1771  of  2004  added
Slide  1772  of  2004  added
Slide  1773  o

RuntimeError: No surface found at the given iso value.

In [3]:
#This code takes each of the tif stacks from a specific folder and saves them as 2D separate images
import os

#Folder with the original images (to get the original names of the slides)
exampleFolder = 'D:/AAV para enfermedades renales/Stardist glomeruli/Stardist results on whole image/' + \
'Result on large image/Kidney1_derecho_4.8x_1x3_R&Lshifted1.6mm'

#Folder where the .tif stacks are
inputFolder = 'D:/AAV para enfermedades renales/Stardist glomeruli/Stardist results on whole image/' + \
'Result on large image/CAM1'

#Folder where you want to store the individual 2D tifs
outputFolder = 'D:/AAV para enfermedades renales/Stardist glomeruli/Stardist results on whole image/' + \
'Result on large image/'+'/Mask-' + os.path.basename(exampleFolder)



from glob import glob as glob
import tifffile as tiff

#List all the .tif stacks in inputFolder
tifStackNames = glob(inputFolder+'/*.tif')

#For each of them
for name in tifStackNames:
    #Read the current .tif stack
    stack = tiff.imread(name)
    print('Stack ',name,' has been opened')
    #Extract the Side, X and Y coordinates from the name:
    stackCoords = 'RL'+name.split('RL')[1].split(' mask')[0]
    #Get the corresponding camera. cam = 'CAM1' or cam = 'CAM2'
    cam = 'CAM'+name.split('CAM')[1][0]
    
    #Original image names
    origImNames = glob(exampleFolder + '/'+ stackCoords+'/*'+cam+'*')
    
    if not os.path.exists(outputFolder + '/' +stackCoords):
        os.makedirs(outputFolder + '/' +stackCoords)
        print("Directory ",outputFolder,'/',stackCoords,' created')
    #For each of the slides in the .tif stack
    for slide in range(stack.shape[0]):
        fileName = outputFolder+'/'+stackCoords+'/'+os.path.basename(origImNames[slide])
        #Save each slide with the same name as the original one
        tiff.imsave(fileName,stack[slide,:,:])
        print('Slide number ',slide,' saved')
    
    

Stack  D:/AAV para enfermedades renales/Stardist glomeruli/Stardist results on whole image/Result on large image/CAM1\Kidney1_derecho_4.8x_1x3_R&Lshifted1.6mmCAM1-Corrected RL00--X00--Y00--C01 mask-z-filtered(dimsZ5Y1X1-conComps.tif  has been opened
Directory  D:/AAV para enfermedades renales/Stardist glomeruli/Stardist results on whole image/Result on large image//Mask-Kidney1_derecho_4.8x_1x3_R&Lshifted1.6mm / RL00--X00--Y00--C01  created
Slide number  0  saved
Slide number  1  saved
Slide number  2  saved
Slide number  3  saved
Slide number  4  saved
Slide number  5  saved
Slide number  6  saved
Slide number  7  saved
Slide number  8  saved
Slide number  9  saved
Slide number  10  saved
Slide number  11  saved
Slide number  12  saved
Slide number  13  saved
Slide number  14  saved
Slide number  15  saved
Slide number  16  saved
Slide number  17  saved
Slide number  18  saved
Slide number  19  saved
Slide number  20  saved
Slide number  21  saved
Slide number  22  saved
Slide number 

Slide number  315  saved
Slide number  316  saved
Slide number  317  saved
Slide number  318  saved
Slide number  319  saved
Slide number  320  saved
Slide number  321  saved
Slide number  322  saved
Slide number  323  saved
Slide number  324  saved
Slide number  325  saved
Slide number  326  saved
Slide number  327  saved
Slide number  328  saved
Slide number  329  saved
Slide number  330  saved
Slide number  331  saved
Slide number  332  saved
Slide number  333  saved
Slide number  334  saved
Slide number  335  saved
Slide number  336  saved
Slide number  337  saved
Slide number  338  saved
Slide number  339  saved
Slide number  340  saved
Slide number  341  saved
Slide number  342  saved
Slide number  343  saved
Slide number  344  saved
Slide number  345  saved
Slide number  346  saved
Slide number  347  saved
Slide number  348  saved
Slide number  349  saved
Slide number  350  saved
Slide number  351  saved
Slide number  352  saved
Slide number  353  saved
Slide number  354  saved


Slide number  648  saved
Slide number  649  saved
Slide number  650  saved
Slide number  651  saved
Slide number  652  saved
Slide number  653  saved
Slide number  654  saved
Slide number  655  saved
Slide number  656  saved
Slide number  657  saved
Slide number  658  saved
Slide number  659  saved
Slide number  660  saved
Slide number  661  saved
Slide number  662  saved
Slide number  663  saved
Slide number  664  saved
Slide number  665  saved
Slide number  666  saved
Slide number  667  saved
Slide number  668  saved
Slide number  669  saved
Slide number  670  saved
Slide number  671  saved
Slide number  672  saved
Slide number  673  saved
Slide number  674  saved
Slide number  675  saved
Slide number  676  saved
Slide number  677  saved
Slide number  678  saved
Slide number  679  saved
Slide number  680  saved
Slide number  681  saved
Slide number  682  saved
Slide number  683  saved
Slide number  684  saved
Slide number  685  saved
Slide number  686  saved
Slide number  687  saved


Slide number  977  saved
Slide number  978  saved
Slide number  979  saved
Slide number  980  saved
Slide number  981  saved
Slide number  982  saved
Slide number  983  saved
Slide number  984  saved
Slide number  985  saved
Slide number  986  saved
Slide number  987  saved
Slide number  988  saved
Slide number  989  saved
Slide number  990  saved
Slide number  991  saved
Slide number  992  saved
Slide number  993  saved
Slide number  994  saved
Slide number  995  saved
Slide number  996  saved
Slide number  997  saved
Slide number  998  saved
Slide number  999  saved
Slide number  1000  saved
Slide number  1001  saved
Slide number  1002  saved
Slide number  1003  saved
Slide number  1004  saved
Slide number  1005  saved
Slide number  1006  saved
Slide number  1007  saved
Slide number  1008  saved
Slide number  1009  saved
Slide number  1010  saved
Slide number  1011  saved
Slide number  1012  saved
Slide number  1013  saved
Slide number  1014  saved
Slide number  1015  saved
Slide num

Slide number  1296  saved
Slide number  1297  saved
Slide number  1298  saved
Slide number  1299  saved
Slide number  1300  saved
Slide number  1301  saved
Slide number  1302  saved
Slide number  1303  saved
Slide number  1304  saved
Slide number  1305  saved
Slide number  1306  saved
Slide number  1307  saved
Slide number  1308  saved
Slide number  1309  saved
Slide number  1310  saved
Slide number  1311  saved
Slide number  1312  saved
Slide number  1313  saved
Slide number  1314  saved
Slide number  1315  saved
Slide number  1316  saved
Slide number  1317  saved
Slide number  1318  saved
Slide number  1319  saved
Slide number  1320  saved
Slide number  1321  saved
Slide number  1322  saved
Slide number  1323  saved
Slide number  1324  saved
Slide number  1325  saved
Slide number  1326  saved
Slide number  1327  saved
Slide number  1328  saved
Slide number  1329  saved
Slide number  1330  saved
Slide number  1331  saved
Slide number  1332  saved
Slide number  1333  saved
Slide number

...

Slide number  543  saved
Slide number  544  saved
Slide number  545  saved
Slide number  546  saved
Slide number  547  saved
Slide number  548  saved
Slide number  549  saved
Slide number  550  saved
Slide number  551  saved
Slide number  552  saved
Slide number  553  saved
Slide number  554  saved
Slide number  555  saved
Slide number  556  saved
Slide number  557  saved
Slide number  558  saved
Slide number  559  saved
Slide number  560  saved
Slide number  561  saved
Slide number  562  saved
Slide number  563  saved
Slide number  564  saved
Slide number  565  saved
Slide number  566  saved
Slide number  567  saved
Slide number  568  saved
Slide number  569  saved
Slide number  570  saved
Slide number  571  saved
Slide number  572  saved
Slide number  573  saved
Slide number  574  saved
Slide number  575  saved
Slide number  576  saved
Slide number  577  saved
Slide number  578  saved
Slide number  579  saved
Slide number  580  saved
Slide number  581  saved
Slide number  582  saved


Slide number  872  saved
Slide number  873  saved
Slide number  874  saved
Slide number  875  saved
Slide number  876  saved
Slide number  877  saved
Slide number  878  saved
Slide number  879  saved
Slide number  880  saved
Slide number  881  saved
Slide number  882  saved
Slide number  883  saved
Slide number  884  saved
Slide number  885  saved
Slide number  886  saved
Slide number  887  saved
Slide number  888  saved
Slide number  889  saved
Slide number  890  saved
Slide number  891  saved
Slide number  892  saved
Slide number  893  saved
Slide number  894  saved
Slide number  895  saved
Slide number  896  saved
Slide number  897  saved
Slide number  898  saved
Slide number  899  saved
Slide number  900  saved
Slide number  901  saved
Slide number  902  saved
Slide number  903  saved
Slide number  904  saved
Slide number  905  saved
Slide number  906  saved
Slide number  907  saved
Slide number  908  saved
Slide number  909  saved
Slide number  910  saved
Slide number  911  saved


Slide number  1193  saved
Slide number  1194  saved
Slide number  1195  saved
Slide number  1196  saved
Slide number  1197  saved
Slide number  1198  saved
Slide number  1199  saved
Slide number  1200  saved
Slide number  1201  saved
Slide number  1202  saved
Slide number  1203  saved
Slide number  1204  saved
Slide number  1205  saved
Slide number  1206  saved
Slide number  1207  saved
Slide number  1208  saved
Slide number  1209  saved
Slide number  1210  saved
Slide number  1211  saved
Slide number  1212  saved
Slide number  1213  saved
Slide number  1214  saved
Slide number  1215  saved
Slide number  1216  saved
Slide number  1217  saved
Slide number  1218  saved
Slide number  1219  saved
Slide number  1220  saved
Slide number  1221  saved
Slide number  1222  saved
Slide number  1223  saved
Slide number  1224  saved
Slide number  1225  saved
Slide number  1226  saved
Slide number  1227  saved
Slide number  1228  saved
Slide number  1229  saved
Slide number  1230  saved
Slide number

Slide number  1509  saved
Slide number  1510  saved
Slide number  1511  saved
Slide number  1512  saved
Slide number  1513  saved
Slide number  1514  saved
Slide number  1515  saved
Slide number  1516  saved
Slide number  1517  saved
Slide number  1518  saved
Slide number  1519  saved
Slide number  1520  saved
Slide number  1521  saved
Slide number  1522  saved
Slide number  1523  saved
Slide number  1524  saved
Slide number  1525  saved
Slide number  1526  saved
Slide number  1527  saved
Slide number  1528  saved
Slide number  1529  saved
Slide number  1530  saved
Slide number  1531  saved
Slide number  1532  saved
Slide number  1533  saved
Slide number  1534  saved
Slide number  1535  saved
Slide number  1536  saved
Slide number  1537  saved
Slide number  1538  saved
Slide number  1539  saved
Slide number  1540  saved
Slide number  1541  saved
Slide number  1542  saved
Slide number  1543  saved
Slide number  1544  saved
Slide number  1545  saved
Slide number  1546  saved
Slide number

Slide number  1825  saved
Slide number  1826  saved
Slide number  1827  saved
Slide number  1828  saved
Slide number  1829  saved
Slide number  1830  saved
Slide number  1831  saved
Slide number  1832  saved
Slide number  1833  saved
Slide number  1834  saved
Slide number  1835  saved
Slide number  1836  saved
Slide number  1837  saved
Slide number  1838  saved
Slide number  1839  saved
Slide number  1840  saved
Slide number  1841  saved
Slide number  1842  saved
Slide number  1843  saved
Slide number  1844  saved
Slide number  1845  saved
Slide number  1846  saved
Slide number  1847  saved
Slide number  1848  saved
Slide number  1849  saved
Slide number  1850  saved
Slide number  1851  saved
Slide number  1852  saved
Slide number  1853  saved
Slide number  1854  saved
Slide number  1855  saved
Slide number  1856  saved
Slide number  1857  saved
Slide number  1858  saved
Slide number  1859  saved
Slide number  1860  saved
Slide number  1861  saved
Slide number  1862  saved
Slide number

In [1]:
#Function to check current variables
%whos

#Check RAM usage
import psutil
d = dict(psutil.virtual_memory()._asdict())
for a in d:
    if a in ['total','available','used','free']:
        d[a] = d[a]/(1024**3)
 
print('\n\nRAM data (Gb)')
print(d)

Interactive namespace is empty.


RAM data (Gb)
{'total': 127.92353820800781, 'available': 121.92282485961914, 'percent': 4.7, 'used': 6.000713348388672, 'free': 121.92282485961914}
